# Import necessary libraries
SAFEP_parse.py contains all the functions and library calls necessary to run the notebook
# Required libraries:
- numpy
- pandas
- matplotlib
- alchemlyb (pip install git+https://github.com/alchemistry/alchemlyb)
- natsort (for sorting file names)
- glob (for unix-like file paths)



# IMPORTANT: Make sure the temperature (set below) matches the temperature you used to run the simulations.

In [ ]:
%cd ..
%pwd

In [ ]:
import warnings
import logging
import copy
from AFEP_parse import *
import matplotlib as mpl
plt.rcParams['figure.dpi'] = 150

logging.captureWarnings(True)
warnings.simplefilter(action='ignore', category=FutureWarning)
#logger = logging.getLogger("alchemlyb.parsers.NAMD")
#logging.disable(level='CRITICAL')

In [ ]:
def checkPaths(paths, nDone):
    goodpaths = []
    for path in paths:
        feps = glob(path+pattern)
        countDone = 0
        for fep in feps:
            with open(fep) as f:
                if 'Free' in f.read():
                    countDone+=1
        if countDone == nDone:
            goodpaths.append(path)
    
    return goodpaths

In [ ]:
def processLeg(paths, RT, decorrelate, pattern, temperature, detectEQ):
    u_nks, cumulative, perWindow, affix = batchProcess(paths, RT, decorrelate, pattern, temperature, detectEQ)

    perWindow[('mean', 'df')] = np.mean(perWindow.loc[:, (slice(None), 'df')], axis=1)
    perWindow[('mean', 'ddf')] = np.mean(perWindow.loc[:, (slice(None), 'ddf')], axis=1)
    perWindow[('mean', 'dG_f')] = np.mean(perWindow.loc[:, (slice(None), 'dG_f')], axis=1)
    perWindow[('mean', 'dG_b')] = np.mean(perWindow.loc[:, (slice(None), 'dG_b')], axis=1)

    reps = list(set(perWindow.columns.get_level_values(0)))
    for key in reps:
        perWindow[(key, 'diff')] = perWindow[(key, 'dG_f')]+perWindow[(key, 'dG_b')]

    keys = set(cumulative.columns.get_level_values(0))
    colors = ['#0072B2', '#D55E00', '#CC79A7', '#009E73', '#E69F00', '#56B4E9']
    keyColors = {}
    i = 0
    for key in keys:
        keyColors[key] = colors[i]
        i += 1

    return reps, keyColors, u_nks, cumulative, perWindow, affix

In [ ]:
def processAllLegs(root, prefixes, nDone=41, checkReplicas=False, dry=False):
    meta_unks = {}
    meta_cumulative = {}
    meta_perWindow = {}
    meta_affix = {}
    meta_keyColors = {}
    meta_reps = {}
    feps = np.round(np.linspace(0, 1, nDone),3)
    meta_fs = {}
    meta_bs = {}
    meta_bes = {}
    meta_fes = {}
    for prefix in prefixes:
        #print(root)
        #print(prefix)
        #print(glob(root+'/'+prefix+'*/'))
        paths = glob(root+prefix+'*/')
        if checkReplicas==True:
            paths = checkPaths(paths, nDone)
        if len(paths)==0:
            print(f"WARNING: Skipping {root+prefix}")
        elif not dry:
            print(paths)
            reps, keyColors, u_nks, cumulative, perWindow, affix = processLeg(paths, RT, decorrelate, pattern, temperature, detectEQ)

            meta_reps[prefix] = reps
            meta_keyColors[prefix] = keyColors
            meta_unks[prefix] = u_nks
            meta_cumulative[prefix] = cumulative
            meta_perWindow[prefix] = perWindow
            meta_affix[prefix] = affix

            fs = {}
            bs = {}
            fes = {}
            bes = {}

            for l in [x for x in meta_reps[prefix] if x != "mean"]:
                u_nks = meta_unks[prefix]
                forward, forward_error, backward, backward_error = doConvergence(u_nks[l], feps)
                fs[l] = forward
                fes[l] = forward_error
                bs[l] = backward
                bes[l] = backward_error

            meta_fs[prefix]=fs
            meta_bs[prefix]=bs
            meta_fes[prefix]=fes
            meta_bes[prefix]=bes
    
    return {'unks':meta_unks, 'cumulatives':meta_cumulative, 'perWins':meta_perWindow, 'affixes':meta_affix, 'keyColors':meta_keyColors, 'reps':meta_reps, 'fs':meta_fs, 'bs':meta_bs, 'fes':meta_fs, 'bes':meta_bes}

In [ ]:
## Membranes
root = '/home/ems363/Documents/ELIC_DCDs_Analyses/'
#root = '/home/ezry/winHome/Documents/ELIC_Data/ELIC/'
systems = ['PCPGPE211', 'PCPG31', 'POPC120']
longSysNames = {'PCPGPE211':r'$\rm2_{POPC}:1_{POPE}:1_{POPG}$', 'PCPG31':r'$\rm3_{POPC}:1_{POPG}$', 'POPC120':'POPC'}

prefixes = ['POCE', 'POCG', 'POEG', 'POGE']
longNames = {'POCE':'POPC to POPE', 'POCG':'POPC to POPG', 'POEG':'POPE to POPG', 'POGE':'POPG to POPE'}

pattern = 'PO*fepout'
checkReplicas = True

In [ ]:
temperature = 303.15
RT = 0.00198720650096 * temperature # ca. 0.59kcal/mol
decorrelate = True #Flag for decorrelation of samples
detectEQ = False #Flag for automated equilibrium detection
DiscrepancyFitting = 'LS' #ML = fit PDF of discrepancies with a normal distribution maximum likelihood estimator. LS = fit CDF of discrepancies with a normal distribution least-squares estimator

In [ ]:
allSys = {}

for sys in systems:
    allSys[sys] = processAllLegs(root+sys+'/', prefixes, 41, checkReplicas=False, dry=False)

In [ ]:
allSys['PCPGPE211'] = processAllLegs(root+'PCPGPE211'+'/', prefixes, 41)

In [ ]:
#Uniquely necessary for PCPG3:1 membrane system

for dat in allSys['PCPG31'].keys():
    allSys['PCPG31'][dat]['POEG'] = copy.deepcopy(allSys['PCPG31'][dat]['POGE'])

In [ ]:
def cumulativePlot(system, leg, keys, keyColors, theax):
    cumulative = system['cumulatives'][leg]
    X = cumulative.index
    for key in keys:
        Y = cumulative[(key, 'f')]*RT
        theColor = keyColors[key]
        theax.plot(X, Y, marker=None, label=key, linewidth=1, color=theColor)
        #finalMean = cumulative.loc[1].iloc[0]*RT
        finalMean = np.mean(cumulative.loc[1, (slice(None), 'f')])
    theax.axhline(finalMean*RT, color='gray', linestyle='-.', linewidth=1)
    
    return theax


def convergencePlot(system, leg, keys, keyColors, theax):
    fs = system['fs'][leg]
    bs = system['bs'][leg]
    for l in [x for x in keys if x != "mean"]:
        theax.plot(np.arange(len(fs[l]))/len(fs[l])+0.1, np.multiply(fs[l], RT), marker='o', linewidth=1, color=keyColors[l], markerfacecolor='white', markeredgewidth=1, markeredgecolor=keyColors[l], ms=5)
        theax.plot(np.arange(len(bs[l]))/len(fs[l])+0.1, np.multiply(bs[l], RT), marker='o', linewidth=1, color=keyColors[l], markerfacecolor='white', markeredgewidth=1, markeredgecolor=keyColors[l], ms=5, linestyle='--')

    theax.xaxis.set_ticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    
    cumulative = system['cumulatives'][leg]
    finalMean = np.mean(cumulative.loc[1, (slice(None), 'f')])
    theax.axhline(y= finalMean*RT, linestyle='-.', color='gray')
    theax.plot(0, finalMean*RT, linewidth=1, color='black', label='Forward Time Sampling')
    theax.plot(0, finalMean*RT, linewidth=1, color='black', linestyle='--', label='Backward Time Sampling')

    #theax.legend()
    
    return theax


def hysteresisPlot(system, leg, keys, keyColors, theax):
    perWindow = system['perWins'][leg]
    # Hysteresis plot
    for key in keys:
        #plt.vlines(perWindow.index, np.zeros(len(perWindow)), perWindow[(key, 'dG_f')] + perWindow[(key, 'dG_b')], label=key, linewidth=2)
        theax.plot(perWindow.index, perWindow[(key, 'diff')]*RT, label=key, linewidth=0.5, alpha=0.8, color=keyColors[key])

    theax.plot(perWindow.index, perWindow[('mean', 'diff')]*RT, label='mean', linewidth=1.5, color='black')

    theax.set(ylim=(-1.25, 1.25))

    return theax

    
def hysteresisPDF(system, leg, keys, keyColors, theax):
    
    perWindow = system['perWins'][leg]
    # Hysteresis PDF
    diff = perWindow.loc[:, (slice(None), 'diff')]*RT
    X = diff.drop(columns='mean') #Including the mean in here would underestimate the skewness of the distribution
    Y = np.arange(len(X))/len(X)
    binNum = 20
    pdfY, pdfX = np.histogram(X, bins=binNum, density=True)
    pdfX = (pdfX[1:]+pdfX[:-1])/2


    #plot the data
    theax.plot(pdfX, pdfY,  label="Estimated Distribution")
    theax.set(xlim=(-1.25, 1.25))
    
    return theax


def makePlots(allSys, systems, prefixes, xlabel, ylabel, doPlot, sharey=True, sharex=True):
    fig, theaxes = plt.subplots(3,len(systems), sharex=sharex, sharey=sharey)
    pad = 5 # in points
    
    for sysID in np.arange(len(systems)):
        sysName = systems[sysID]
        system = allSys[sysName]
        print(f'Now making figures for: {sysName}')
        theaxes[2, sysID].set(xlabel=xlabel)
        theaxes[0, sysID].annotate(longSysNames[sysName], xy=(0.5, 1), xytext=(0, pad),
                    xycoords='axes fraction', textcoords='offset points',
                    size='large', ha='center', va='baseline')
        for prefID in np.arange(len(prefixes)):
            # Cumulative change in kT 
            leg = prefixes[prefID]
            try:
                keys = system['keyColors'][leg]
                keyColors = system['keyColors'][leg]
                theaxes[prefID, 0].set(ylabel=ylabel)
                theaxes[prefID, 0].annotate(longNames[leg], xy=(0, 0.5), xytext=(-theaxes[prefID, 0].yaxis.labelpad - pad, 0),
                        xycoords=theaxes[prefID, 0].yaxis.label, textcoords='offset points',
                        size='large', ha='right', va='center', rotation=90)

                theax = doPlot(system, leg, keys, keyColors, theaxes[prefID, sysID])
            except:
                continue
            
    fig.set_figheight(10)
    fig.set_figwidth(8)
    fig.tight_layout()
    
    return fig, theaxes


In [ ]:
fig, theaxes = makePlots(allSys, systems, prefixes, xlabel=r'$\rm\delta_{\lambda}$ (kcal/mol)', ylabel='PDF', doPlot=hysteresisPDF, sharey=True)
plt.savefig(f"{root}/DRAFT_S3.pdf")

In [ ]:
fig, theaxes = makePlots(allSys, systems, prefixes, xlabel=r'$\rm\lambda$', ylabel=r'$\rm\delta_{\lambda}$ (kcal/mol)', doPlot=hysteresisPlot, sharey='row')
plt.savefig(f"{root}/DRAFT_S2.pdf")

In [ ]:
for x in allSys['PCPG31']['fs']['POGE'].keys():
    allSys['PCPG31']['fs']['POEG'][x] = np.multiply(copy.deepcopy(allSys['PCPG31']['fs']['POGE'][x]), -1)
    
for x in allSys['PCPG31']['bs']['POGE'].keys():
    allSys['PCPG31']['bs']['POEG'][x] = np.multiply(copy.deepcopy(allSys['PCPG31']['bs']['POGE'][x]), -1)
                                                    

In [ ]:
fig, theaxes = makePlots(allSys, systems, prefixes, xlabel='Fraction of simulation time', ylabel=r'$\rm\Delta G (kcal/mol)$', doPlot=convergencePlot, sharey='row')
plt.savefig(f"{root}/DRAFT_S1.pdf")

In [ ]:
for x in allSys['PCPG31']['cumulatives']['POEG'].keys():
    baseline = allSys['PCPG31']['cumulatives']['POGE'][x].iloc[-1]*-1
    allSys['PCPG31']['cumulatives']['POEG'][x] = (copy.deepcopy(allSys['PCPG31']['cumulatives']['POGE'][x]) + baseline)
    temp = np.flip(allSys['PCPG31']['cumulatives']['POEG'][x])
    temp.index = allSys['PCPG31']['cumulatives']['POEG'][x].index
    
    allSys['PCPG31']['cumulatives']['POEG'][x] = copy.deepcopy(temp)

In [ ]:
allSys['PCPGPE211']['keyColors']['POEG']['POEG_Good'] = '#000000'

In [ ]:
fig, dGaxes = makePlots(allSys, systems, prefixes, xlabel=r'$\rm\lambda$', ylabel=fr'Cumulative $\rm\Delta$G($\rm\lambda$) (kcal/mol)', doPlot=cumulativePlot)
plt.savefig(f"{root}/DRAFT_S4.pdf")